# LLMS

In [ ]:
from config import *

In [2]:
from langchain.llms import OpenAI
llm = OpenAI()

In [3]:
llm("On what day is Pakistan's Independence Day celebrated?")

"\n\nPakistan's Independence Day is celebrated on August 14th."

In [4]:
llm_results = llm.generate(["Tell me a joke", "Tell me a poem"]*3)

In [14]:
print(llm_results.generations[0][0].text)



Q: What did the fish say when it hit the wall?
A: Dam!


In [15]:
llm_results.llm_output

{'token_usage': {'prompt_tokens': 24,
  'total_tokens': 388,
  'completion_tokens': 364},
 'model_name': 'text-davinci-003'}

# Async API

In [18]:
import time
import asyncio

from langchain.llms import OpenAI

def generate_serially():
    llm = OpenAI(temperature=0.9)
    for _ in range(10):
        resp = llm.generate(["Hello, how are you?"])
        print(resp.generations[0][0].text)
        
async def async_generate(llm):
    resp = await llm.agenerate(["Hello, how are you?"])
    print(resp.generations[0][0].text)
    
async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    tasks = [async_generate(llm) for _ in range(10)]
    await asyncio.gather(*tasks)
    
s = time.perf_counter()

await generate_concurrently()

elapsed = time.perf_counter() - s

print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

s = time.perf_counter()

generate_serially()
elapsed = time.perf_counter() - s

print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")



I'm doing great, thank you. How about yourself?


I'm doing well, thank you. How about you?


I'm doing well. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thanks. How about yourself?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about yourself?


I'm doing well, thanks for asking. How about you?


I'm doing well, thank you for asking. How about you?


I'm doing well, thank you. How about you?
Concurrent executed in 1.30 seconds.


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you for asking. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing alright, and yourself?
Concurrent executed in 8.48 seconds.


# Custom LLM

In [19]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

In [ ]:
class CustomLLM(LLM):
    n: int
    
    @property
    def _llm_type(self) -> str:
        return "CustomLLM"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        
        if stop is not None:
            raise ValueError("stop is not supported for CustomLLM")
        return prompt[: self.n]